# Ana Bileşenler Analizi, ABA (Principal Component Analysis, PCA)
Hazırlayan: Marc Deisenroth ve Yicheng Luo
Çeviren: Utku Karaca

ABA algoritmasını izdüşüm perspektifini kullanarak uygulayacağız. Öncelikle algoritmayı kodlayacağız, ardından MNIST rakam veri kümesine uygulayacağız.

Gerekli paketler:

In [ ]:
# nümerik işlemler kütüphanesi
import numpy as np
# görsel kütüphanesi
import matplotlib as mpl
mpl.use('Agg')
# görsel şekil ayarı
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from ipywidgets import interact
# veri kümesi
from sklearn.datasets import fetch_openml
images, labels = fetch_openml('mnist_784', version=1, return_X_y = True)
# görsel gösterim ayarı
%matplotlib inline

Gözlem sayısı:

In [ ]:
print("Gözlem sayısı:",len(images))

Veri seti:

In [ ]:
print(images.head())
labels

Örnek görsel:

In [ ]:
plt.figure(figsize=(4,4))
plt.imshow(np.array(images.iloc[0]).reshape(28,28), cmap='gray')
print('Rakam:', labels[0])

# ABA
ABA kodlamasına geçmeden önce veri seti üzerinde bazı işlemler yapmalıyız. 
1. Piksel değerlerini 0 ve 1 aralığına çevirmek
2. Her bir görselden ortalamayı ($\boldsymbol \mu$) çıkarmak
3. Her bir görselin her bir boyutunu standart sapmaya ($\sigma$) göre ölçeklemek ($\frac{1}{\sigma}$).

Bu adımlar bütün görsellerin ortalamasının sıfır ve varyansının bir olmasını sağlayacak, standarlaştıracak.

In [ ]:
def standartlastir(X):
    """ Verilen veri kümesi standartlaştırır
    Parametreler:
        X: ndarray, veri kümesi
    
    Dön:
        (Xbar, ortalama, std): Xbar standantlaştırılmış veri kümesi.
        Ortalaması 0, standart sapması 1.
    
    Not:
        Standart sapmanın 0 olacağı yerlerde sorun çıkmaması için 0
        0 olan yerlerin standart sapma değerini 1 yapacağız.
    """
    ortalama = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    std_degistirilmis = std.copy()
    std_degistirilmis[std==0] = 1.
    Xbar = ((X-ortalama)/std_degistirilmis)
    return Xbar, ortalama, std

def oz(S):
    """ Kovaryans matrisi S için özdeğer ve özvektörleri hesaplar. 
    
    Parametreler:
        S: ndarray, kovaryans matrisi
    
    Dön:
        (ozdegerler, ozvektorler): ndarray
        
    Not:
        özdeğerler ve özvektörler özdeğerlerin azaldığı sırayla
        sıralanmalıdır.
    """
    ozdegerler, ozvektorler = np.linalg.eig(S)
    k = np.argsort(ozdegerler)[::-1] #[<start>:<stop>:<step>] ozdegerler'i küçükten büyüğe sırayıp, ters çevirme
    return ozdegerler[k], ozvektorler[:,k]

def izdusum_matrisi(B):
    """ `B` tarafından kapsanan uzay üzerine izdüşüm matrisini
    hesaplar
    
    (Compute the projection matrix onto the space spanned by `B`)
    Parametreler:
        B: ndarray, (D, M), altuzay tabanları
    
    Dön:
        P: izdüşüm matrisi, B*(B.T*B)^-1*B.T, B*B.T
    """
    return (B.dot(np.linalg.inv(B.T.dot(B))).dot(B.T)) #(10.34), alternatif, return B.dot(B.T)

def ABA(X, bilesen_sayisi):
    """
    Parametreler:
        X: standartlaştırılmış, ndarray, (N, D), D veri kümesinin boyutu,
           N  veri kümesindeki veri sayısı.
        bilesen_sayisi: kullanılacak ana bileşen sayısı
    Dön:
        X_yeni: bilesen_sayisi kadar bileşenle oluşturulan 
        yeni X matrisi
    """
    # öncelikle standartlaştırma yapmalıyız
    # sonra veri kovaryans matrisi S'i hesaplamalıyız
    S = 1.0/len(X) * np.dot(X.T, X)

    # S için özdeğerler ve özvektörleri bulma
    ozdegerler, ozvektorler = oz(S)
    # en büyük özdeğerlerin yerlerini bulup 
    # özvektörleri sıralamak için kullanacağız.
    
    # bileşen sayısı kadarını seçmeliyiz
    ozdegerler, ozvektorler = ozdegerler[:bilesen_sayisi], ozvektorler[:, :bilesen_sayisi]

    # Gerçek veri kümesinin boyutunu indirgeme
    B = np.real(ozvektorler)
    # Z = X.T.dot(W)
    # görselleri düşük boyut temsilleriyle tekrar oluşturma
    X_yeni = (izdusum_matrisi(B).dot(X.T)) #(10.32)
    return X_yeni.T

In [ ]:
## Veri yaratımı ve önişleme adımları
veri_sayisi = 1000
X = (np.array(images)[:veri_sayisi]) / 255.
Xbar, mu, std = standartlastir(X)

from sklearn.decomposition import PCA as SKPCA
for bilesen_sayisi in range(1, 20):
    # scikit-learn kütüphanesinin ABA kodlamasıyla kendimizinkini karşılaştıralım
    pca = SKPCA(n_components=bilesen_sayisi, svd_solver='full')
    sklearn_yeni = pca.inverse_transform(pca.fit_transform(Xbar))
    yeni = ABA(Xbar, bilesen_sayisi)
    np.testing.assert_almost_equal(yeni, sklearn_yeni)
    print(np.square(yeni - sklearn_yeni).sum())

Soru:
> Ortalama Karesel Hatanın (OKH) 100'den az olması için kaç tane bileşene ihtiyacımız vardır?

In [ ]:
def okh(kestirim, gercek):
    """Ortalama karesel hata (OKH) hesaplar"""
    return np.square(kestirim - gercek).sum(axis=1).mean()

In [ ]:
kayıp = []
yeniler = []

# farklı sayıda ana bileşenleri dener ve OKH değerlerini hesaplar
for bilesen_sayisi in range(1, 100):
    yeni = ABA(Xbar, bilesen_sayisi)
    hata = okh(yeni, Xbar)
    yeniler.append(yeni)
    kayıp.append((bilesen_sayisi, hata))

yeniler = np.asarray(yeniler)
yeniler = yeniler * std + mu # yeniden yaratılan görsel üzerindeki 
                             # standartlaştırma işlemini geri alır
kayıp = np.asarray(kayıp)

import pandas as pd
# ana bileşen sayıları ve OKH değerlerini gösteren tablo oluşturulması
pd.DataFrame(kayıp).head()

# görsel olarak inceleme
fig, ax = plt.subplots()
ax.plot(kayıp[:,0], kayıp[:,1]);
ax.axhline(100, linestyle='--', color='r', linewidth=2)
ax.xaxis.set_ticks(np.arange(1, 100, 5));
ax.set(xlabel='bilesen_sayisi', ylabel='OKH', title='OKH - ana bileşen sayısı');

Fakat sayılar bize her şeyi anlatmazlar. OKH değerindeki bu düşüş ne anlama geliyor?

Bir alttaki kod parçacığında en solda gerçek görsel gösteriliyor. Sağa doğru gittikçe görseller farklı sayıda ana bileşenler kullanılarak tekrar oluşturuluyor.

In [ ]:
@interact(image_idx=(0, 999))
def bilesen_yenileri_goster(image_idx):
    fig, ax = plt.subplots(figsize=(20., 20.))
    gercek = X[image_idx]
    # gerçek görsel ile tekrar oluşturulan görseller yanyana ekleniyor
    x = np.concatenate([gercek[np.newaxis, :], yeniler[:, image_idx]])
    ax.imshow(np.hstack(x.reshape(-1, 28, 28)[np.arange(10)]),
              cmap='gray');
    ax.axvline(28, color='orange', linewidth=2)

Diğer rakamların tekrar oluşturulan görselleri:

In [ ]:
@interact(i=(0, 10))
def aba_rakam_goster(i=1):
    """ i'nci rakamı ve tekrar yaratılmış halini gösterir"""
    plt.figure(figsize=(4,4))
    gercek_ornek = X[i].reshape(28,28)
    yeni_ornek = (yeni[i, :] * std + mu).reshape(28, 28)
    plt.imshow(np.hstack([gercek_ornek, yeni_ornek]), cmap='gray')
    plt.show()